

*   Capture frames from a video file.

*   Use the first 20 frames to populate the history of a background subtractor

*   Based on background subtraction, use the 21st frame to identify moving foregrond objects
*  We will treat these as pedestrians. For each pedestrian, assign an ID and an initial tracking window and then we calculate the histogram


*   LFor each subsequent frame, track each pedestrian uising Kalman filter and Mean shift


Before we start to implement  we need to digest to consider programming paradigms and how they relate to our use of OpenCV.

Another paradigm called FP has been gaining support among programmers who favor a purer mathematical foundation.

FB allows functions to return functions and permits functions as arguments in a function.

Meanshift algorithm performs tracking iteratively by computing a centroid based on probability values in the current tracking rectangle, shifting the rectangle's center to this centroid, recomputing the centroid based values in the new rectangle, shifting the rectangle again and so on.

This process continues until convergence is achieved (meaning that the centroid ceases to move or nearly ceases to move) or until a maximum number of iterations is reach.

Meanshift is a clustering algorithm with applications that extend beyound computer vision.

Kalman filter algorithm is developed mainly by Rudolf Kalman.

Practical applications in navigation systems for all any types of vehicle from nuclear submarine to aircraft.

In computer vision, Kalman filter can smoothen the estimate of a tracked object's position


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/Colab Notebooks

/content/drive/My Drive/Colab Notebooks


In [3]:
from google.colab.patches import cv2_imshow

In [4]:
import cv2
import numpy as np
OPENCV_MAJOR_VERSION = int(cv2.__version__.split('.')[0])

In [ ]:
class Pedestrian():
    """A tracked pedestrian with a state including an ID, tracking
    window, histogram, and Kalman filter.
    """

    def __init__(self, id, hsv_frame, track_window):

        self.id = id

        self.track_window = track_window
        self.term_crit = \
            (cv2.TERM_CRITERIA_COUNT | cv2.TERM_CRITERIA_EPS, 10, 1)

        # Initialize the histogram.
        x, y, w, h = track_window
        roi = hsv_frame[y:y+h, x:x+w]
        roi_hist = cv2.calcHist([roi], [0], None, [16], [0, 180])
        self.roi_hist = cv2.normalize(roi_hist, roi_hist, 0, 255,
                                      cv2.NORM_MINMAX)

        # Initialize the Kalman filter.
        self.kalman = cv2.KalmanFilter(4, 2)
        self.kalman.measurementMatrix = np.array(
            [[1, 0, 0, 0],
             [0, 1, 0, 0]], np.float32)
        self.kalman.transitionMatrix = np.array(
            [[1, 0, 1, 0],
             [0, 1, 0, 1],
             [0, 0, 1, 0],
             [0, 0, 0, 1]], np.float32)
        self.kalman.processNoiseCov = np.array(
            [[1, 0, 0, 0],
             [0, 1, 0, 0],
             [0, 0, 1, 0],
             [0, 0, 0, 1]], np.float32) * 0.03
        cx = x+w/2
        cy = y+h/2
        self.kalman.statePre = np.array(
            [[cx], [cy], [0], [0]], np.float32)
        self.kalman.statePost = np.array(
            [[cx], [cy], [0], [0]], np.float32)

    def update(self, frame, hsv_frame):

        back_proj = cv2.calcBackProject(
            [hsv_frame], [0], self.roi_hist, [0, 180], 1)

        ret, self.track_window = cv2.meanShift(
            back_proj, self.track_window, self.term_crit)
        x, y, w, h = self.track_window
        center = np.array([x+w/2, y+h/2], np.float32)

        prediction = self.kalman.predict()
        estimate = self.kalman.correct(center)
        center_offset = estimate[:,0][:2] - center
        self.track_window = (x + int(center_offset[0]),
                             y + int(center_offset[1]), w, h)
        x, y, w, h = self.track_window

        # Draw the predicted center position as a blue circle.
        cv2.circle(frame, (int(prediction[0]), int(prediction[1])),
                   4, (255, 0, 0), -1)

        # Draw the corrected tracking window as a cyan rectangle.
        cv2.rectangle(frame, (x,y), (x+w, y+h), (255, 255, 0), 2)

        # Draw the ID above the rectangle in blue text.
        cv2.putText(frame, 'ID: %d' % self.id, (x, y-5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0),
                    1, cv2.LINE_AA)

Kalman filter can predict the postion of an object based on historical observations and can correct the prediction based on the actual data. However, it can only do this for 1 object.

That is why we need one Kalman filter for each object which will be tracked.

Tracking window will be used by the MeanShift algorithm

We will display the ID of pedestrians so it will be easier to distinguish between all of the pedestrians